In [ ]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
import pandas as pd

from src.loader import TripLoader
from src.utils import *

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
df_air = pd.read_csv("../src/data/extracted/abcd_UpdateFlightAction_saved.csv")

In [ ]:
df_air2 = pd.read_csv("../src/data/extracted/mnop_UpdateFlightAction_saved.csv")
df_air3 = pd.read_csv("../src/data/extracted/zyxw_UpdateFlightAction_saved.csv")

In [ ]:
df_air.head()

In [ ]:
df_air["legs"] = df_air["legs"].apply(eval)

In [ ]:
filtered_df = df_air[df_air["legs"].apply(len) == 2]

In [ ]:
filtered_df.head()

In [ ]:
len(filtered_df)

In [ ]:
df_air2["legs"] = df_air2["legs"].apply(eval)
df_air3["legs"] = df_air3["legs"].apply(eval)

In [ ]:
filtered_df2 = df_air2[df_air2["legs"].apply(len) == 2]
filtered_df3 = df_air3[df_air3["legs"].apply(len) == 2]

In [ ]:
len(filtered_df2), len(filtered_df3)

In [ ]:
df_all = pd.concat([filtered_df, filtered_df2, filtered_df3])

In [ ]:
df_all.info()

In [ ]:
# extract airport codes from legs
# df_all['origin'] = df_all['legs'].apply(lambda x: x[0]['origin'])

In [ ]:
def extract_airports(legs):
    dep_airports = [leg["DEP"] for leg in legs]
    dest_airports = [leg["DEST"] for leg in legs]
    return dep_airports, dest_airports


# Apply the function to create new columns for departure and destination airports
df_all["first"], df_all["second"] = zip(*df_all["legs"].apply(extract_airports))

# Display the updated DataFrame
df_all.head()

In [ ]:
new_data = []

for index, row in df_all.iterrows():
    flight_id = row["flight_id"]
    id = row["id"]
    Airline = row["Airline"]
    first_leg = row["first"]
    second_leg = row["second"]

    # Create entries for the first leg
    new_data.append(
        [
            flight_id,
            id,
            Airline,
            "DEP",
            first_leg[0],
            f"{first_leg[0]}_{first_leg[1]}",
            f"{first_leg[0]}_{second_leg[1]}",
        ]
    )
    new_data.append(
        [
            flight_id,
            id,
            Airline,
            "ARR",
            first_leg[1],
            f"{first_leg[0]}_{first_leg[1]}",
            f"{first_leg[0]}_{second_leg[1]}",
        ]
    )

    # Create entries for the second leg
    new_data.append(
        [
            flight_id,
            id,
            Airline,
            "DEP",
            second_leg[0],
            f"{second_leg[0]}_{second_leg[1]}",
            f"{first_leg[0]}_{second_leg[1]}",
        ]
    )
    new_data.append(
        [
            flight_id,
            id,
            Airline,
            "ARR",
            second_leg[1],
            f"{second_leg[0]}_{second_leg[1]}",
            f"{first_leg[0]}_{second_leg[1]}",
        ]
    )

# Convert the new data into a DataFrame
columns = ["flight_id", "id", "Airline", "Type", "Airport", "route", "long_flight"]
new_df = pd.DataFrame(new_data, columns=columns)

In [ ]:
new_df.head()

In [ ]:
arr_airports = pd.read_csv("arr_airports.csv")

In [ ]:
arr_airports.head()

In [ ]:
# Merge the new DataFrame with the arr_airports DataFrame with different val names
df_fin = new_df.merge(
    arr_airports, left_on="Airport", right_on="arr_Airport", how="left"
)

In [ ]:
df_fin

In [ ]:
# export to excel
df_fin.to_excel("df_zwischenflug_long.xlsx", index=False)

### DF für weiterführende Diagramme

Zwischenflüge Lande Flughafen

In [ ]:
# take only index with odd index
df_dia = new_df[new_df.index % 4 == 1]

In [ ]:
df_dia.head()

In [ ]:
# group by airport and add airline
df_grouped_dia = df_dia.groupby("Airport").agg({"id": "count"}).reset_index()

In [ ]:
# merge with arr_airports
df_dia_fin = df_grouped_dia.merge(
    arr_airports, left_on="Airport", right_on="arr_Airport", how="left"
)

In [ ]:
df_dia_fin.head()

In [ ]:
df_dia_fin["id"].sum()

In [ ]:
# export to excel
df_dia_fin.to_excel("zwischenlandungen.xlsx", index=False)

In [ ]:
# group by airline
df_grouped_airline = df_dia.groupby("Airline").agg({"id": "count"}).reset_index()

In [ ]:
df_grouped_airline

In [ ]:
# export to excel
df_grouped_airline.to_excel("airline_zwischenlandungen.xlsx", index=False)